In [1]:
# pip install scrapy
import scrapy

In [ ]:

# course : https://www.youtube.com/watch?v=mBoX_JCKZTE

# to start scraping project, go to command line : 
    # scrapy startproject project_name
        # you will see that 'spider' folder and many other files are created in the directory. 
        # 'spider' folder contains files for spiders(scrapy crawlers)
        # pipelines.py for pipelines, etc
        # settings.py for settings of the crawler. you can change however you want
        # scrapy.cfg is a configuration file. 

# navigate to 'spiders' folder in command line then, we create a spider using the following command : 
    # scrapy genspider spider_name url_to_scrape
# now, you will see a spider_name.py file inside 'spiders' directory


# if you want to use ipthon shell instead of default scrapy shell, you can do the following:
#  install ipython(pip install ipython) and in ../scrapy.cfg file, you need to add set shell as 'ipython'[shell = ipython ---> add this line in scrapy.cfg file] 
# you can use scrapy shell to run commands before impementing it in the script.(command to start the shell : scrapy shell)

# to start crawling : 
    # scrapy crawl spider_name


# to initialize items, have to import the class that is declared in the 'items.py' file in the spider script.
# items.py has a class that has variables to hold the scraped values. 

# we can use pipelines(pipelines.py) for preprocessing the data. you will also need to uncomment the 'ITEM_PIPELINES' variable from settings.py file and check if the pipeline name inside the variable matches the class name of your pipeline otherwise you will have errors.


# saving the scraped data:
    # via command line  : scrapy crawl spider_name -O file_name.extension -> rewrite the file, if already exists. but if :     scrapy crawl spider_name -o file_name.extension -> append the data, if file already exists
    # via feed settings : add the following lines in the 'settings.py' file : 
            # FEEDS = {
            #           'filename.extension' : {'format' : 'extension'}   
            # }
                        # filename.extension will be created with the scraped data.

            """   
                    you can also provide custom setting in the spider script file itself. just add 'custom_settings' dictionary containing all the settings.
                    for example : 
                        custrom_settings = {
                            'FEEDS' : {'filename.extension' : {'format' : 'extension', 'overwrite' : True/False}}   #if set True, then overwrite the settings in settings.py file if this setting is already present in the file.
                        }   
            """
    # saving the data into databases from pipelines.py file : see pipelines.py file.



    # fake user agents and request headers : many websites try to block the bots by seeing our request headers(we can see out request headers under 'network' section inside inspect elements in our browser. go to network section and reload the webpage and click on the url and scroll down you will see the request headers inside headers section and you will see the user agent at the bottom of the request headers. you can decode the user agent by pasting the user agent value in https://useragentstring.com/)
        # setting our own user agent value : 
            # go to settings.py and enter :
                # USER_AGENT = 'valid useragent string' example : USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
            # but this method is useless as it doesnt prevent us from getting bolocked by website because it doesnt change the user agent value.

        # so in order to avoid getting blocked, one thing we can do is change the request headers and user agents at every request / at every 'n' requests.
            # first, create a list of fake user agents in your spider script.(you can download a list of fake user agents from internet)
            # then, at wherever you are making a request , you need to specify the user agent value. for example, see 'adv_spider.py' spider script.

        # getting thousands of random user agents and request headers from https://scrapeops.io/ to scrape massive websites like amazon.com, etc..
                # login credentials : username : 537fm7dn@duck.com
                                    # password : p@$$w0rd
                                    # api key : 57d1e6ca-6e1f-4cd2-8f53-f2ce027ad9f2
                # for fake user agents : we have created a class(getuseragents) in middlewares.py. to enable this middleware you have to uncomment the 'DOWNLOADER_MIDDLEWARES' variable in 'settings.py' and add your class in the variable. (for example see 'adv_spider_sub1.py' spider script and see 'getuseragents' class in 'middlewares.py')

                # for fake response headers : we have created a class(getrequestheaders) in middlewares.py. to enable this middleware you have to uncomment the 'DOWNLOADER_MIDDLEWARES' variable in 'settings.py' and add your class in the variable. (for example see 'adv_spider_sub1.py' spider script and see 'getrequestheaders' class in 'middlewares.py')
                        # you can also integrate the scrapeops api directly into the spider script also where we can spin up new fake browser headers and user-agents during each iteration.

        # the website can also detect the bot if we use the same ip address to scrape. so we can use proxys to avoid getting blocked
                # install scrapy-rotating-proxies(pip install scrapy-rotating-proxies)- middleware for changing proxies while scraping
                #first, we have prepared a list of proxies in settings.py. make  sure that the variable name where you have stored the proxies is . these proxies are taken from 'https://free-proxy-list.net/' and 'https://geonode.com/free-proxy-list'.
                # then, go ahead and add these middlewares in the 'DOWNLOADER_MIDDLEWARES' variable
                        # "rotating_proxies.middlewares.RotatringProxyMiddleware" : n,  
                        # "rotating_proxies.middlewares.BanDetectionMiddleware" : n+1      here, n => integer
                # if you have saved list of proxies in a file, then save the path of the file in 'ROTATING_PROXY_LIST_PATH' variable in 'settings.py' file

                # NOTE :  the main disadvantage of the above method is that these free proxies are used my others also so they will be very slow. you can get your own deticated proxies from 'https://smartproxy.com/'. then you can either implement it directly into your spider script by passing "meta = {'proxy' : url}" in scrapy.Request() or response.follow(). 
                    # example : 
                            # yield scrapy.Request(relative_url, callback = self.parse_book_page, meta = {'proxy' : url})

                    #  or else you can also create a class in middleware and change your proxies there.
                        # example : 
                                # first, all the following lines in the 'settings.py' file
                                    ## settings.py
                                        # PROXY_USER = 'username'
                                        # PROXY_PASSWORD = 'password'
                                        # PROXY_ENDPOINT = 'gate.smartproxy.com'
                                        # PROXY_PORT = '7000'

                                        # DOWNLOADER_MIDDLEWARES = { 
                                        #     'bookscraper.middlewares.MyProxyMiddleware': 350, 
                                        #     'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 400, 
                                        # }
                                # then, create a class in 'middlewares.py' file and add this class : 
                                        import base64

                                        # class MyProxyMiddleware(object):

                                        #     @classmethod
                                        #     def from_crawler(cls, crawler):
                                        #         return cls(crawler.settings)

                                        #     def __init__(self, settings):
                                        #         self.user = settings.get('PROXY_USER')
                                        #         self.password = settings.get('PROXY_PASSWORD')
                                        #         self.endpoint = settings.get('PROXY_ENDPOINT')
                                        #         self.port = settings.get('PROXY_PORT')

                                        #     def process_request(self, request, spider):
                                        #         user_credentials = '{user}:{passw}'.format(user=self.user, passw=self.password)
                                        #         basic_authentication = 'Basic ' + base64.b64encode(user_credentials.encode()).decode()
                                        #         host = 'http://{endpoint}:{port}'.format(endpoint=self.endpoint, port=self.port)
                                        #         request.meta['proxy'] = host
                                        #         request.headers['Proxy-Authorization'] = basic_authentication

                        # we have implemented proxies from scrapeops.io api. see adv_spider_sub2.py 
                                #  to implement the same using middle wares, see adv_spider_sub2_pt2.py script
                                        # for this install scrapeops-scrapy-proxy-sdk(pip install scrapeops-scrapy-proxy-sdk)

                        # running spider in cloud :  refer : https://www.youtube.com/watch?v=mBoX_JCKZTE
                            # using scrapyd
                            # using scrapeops
                            # using scrapy cloud


